# Importing

In [1]:
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup 
import re
import pandas as pd 
import os 
import time
import datetime
import csv
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from __future__ import division

# Reading the 3 workouts dfs to clean

In [2]:
apr14_workouts = pd.read_csv('workouts_csv')
apr7_workouts = pd.read_csv('workouts_csv_2')
apr6_workouts = pd.read_csv('workouts_csv_3')

In [3]:
allworkouts_uncleaned = pd.concat([apr14_workouts, apr7_workouts, apr6_workouts], axis=0)

In [4]:
allworkouts_uncleaned.to_csv('allworkouts_uncleaned')

In [5]:
len(allworkouts_uncleaned)

2164

In [6]:
allworkouts_uncleaned.sample(10)

,Unnamed: 0,Course,Date,Distance,Horse Name,Note,Rank,Time_tenths_second,Track
67,67,Dirt,03/04/2018,4F,Sellwood,Handily,\n 22/34 \n ...,488,Santa Anita
689,689,Dirt training,12/28/2016,4F,Play Hard to Get,Handily,\n 5/6 \n ...,496,Santa Anita
497,497,Dirt,07/29/2016,5F,Lord Admiral,Handily,\n 2/5 \n ...,622,San Luis Rey Training Center
646,646,Dirt,08/28/2016,5F,River Echo,Breezing,\n 4/32 \n ...,606,Saratoga
23,23,Dirt,04/22/2017,3F,Blame the Rider,Handily,\n 4/21 \n ...,366,Santa Anita
14,14,Dirt,10/06/2017,6F,Blame the Rider,Handily,\n 6/19 \n ...,744,Santa Anita
13,13,Dirt,12/03/2017,6F,Accountability,Handily,\n 6/6 \n ...,752,Santa Anita
460,460,Dirt,07/13/2017,4F,Croissant,Handily,\n 5/17 \n ...,482,Los Alamitos Quarter Horse
594,594,Dirt training,07/17/2016,4F,Sassy Little Lila,Breezing,\n 45/90 \n ...,495,Belmont Park
217,217,Dirt,04/02/2018,5F,Papa Joe,Handily,\n 72/81 \n ...,632,Santa Anita


In [7]:
allworkouts_clean = allworkouts_uncleaned.drop('Unnamed: 0', axis=1).drop('Course', axis=1).drop('Note', axis=1 ).drop('Rank', axis=1).drop('Track',axis=1)



In [8]:
allworkouts_clean

,Date,Distance,Horse Name,Time_tenths_second
0,04/07/2018,5F,Blame the Rider,618
1,03/31/2018,5F,Blame the Rider,616
2,03/24/2018,4F,Blame the Rider,474
3,03/01/2018,4F,Blame the Rider,490
4,02/23/2018,5F,Blame the Rider,614
5,02/16/2018,4F,Blame the Rider,512
6,01/14/2018,5F,Blame the Rider,610
7,12/18/2017,6F,Blame the Rider,752
8,12/11/2017,5F,Blame the Rider,620
9,12/04/2017,6F,Blame the Rider,730


In [37]:
Practice_days = allworkouts_clean.groupby(['Horse Name']).Date.count().reset_index()
#Practice_days = allworkouts_clean.groupby((['Horse Name']).Date isin '\17')

#Practice_counts = allworkouts_clean.groupby(['Horse Name']).Distance.value_counts()
#allworkouts_clean3 = allworkouts_clean2[(allworkouts_clean2.Distance == "4F") | (allworkouts_clean2.Distance == "5F") 

In [38]:
Practice_days.head(10) ###Combine into final data frame. 

,Horse Name,Date
0,Accountability,59
1,Air Vice Marshal,38
2,Aquaphobia,47
3,Avalanche,52
4,Beach View,41
5,Beau Recall,46
6,Beauty Divine,27
7,Blame the Rider,28
8,Blossom Trail Miss,51
9,Cafe Flavor,33


In [11]:
#allworkouts_clean = allworkouts_clean.filter['5F','4F']

#allworkouts_clean = (allworkouts_clean['Distance'] == '5F') or (allworkouts_clean['Distance'] == '4F')

#FiveF = allworkouts_clean['Distance'] != "2F","3F","6F"
#FourF = allworkouts_clean['Distance'] == "4F"
#allworkouts_clean = allworkouts_clean[FiveF]


### Creating df from workouts df of number of practice days

In [12]:
allworkouts_clean2 = allworkouts_clean.groupby(['Horse Name', "Distance"]).Time_tenths_second.agg([np.mean]).reset_index()
allworkouts_clean3 = allworkouts_clean2[(allworkouts_clean2.Distance == "4F") | (allworkouts_clean2.Distance == "5F") | (allworkouts_clean2.Distance == "3F")]
allworkouts_clean3.head(10)

,Horse Name,Distance,mean
0,Accountability,3F,366.800000
1,Accountability,4F,491.052632
2,Accountability,5F,613.363636
5,Air Vice Marshal,3F,369.400000
6,Air Vice Marshal,4F,488.538462
7,Air Vice Marshal,5F,616.066667
10,Aquaphobia,3F,385.000000
11,Aquaphobia,4F,496.652174
12,Aquaphobia,5F,618.421053
14,Avalanche,3F,368.400000


In [13]:
allworkouts_clean4 = allworkouts_clean3.pivot(index='Horse Name', columns='Distance', values='mean').reset_index()


#allworkouts_clean4 = allworkouts_clean3.unstack(level=-1)

In [14]:
allworkouts_clean4.head(10)

Distance,Horse Name,3F,4F,5F
0,Accountability,366.800000,491.052632,613.363636
1,Air Vice Marshal,369.400000,488.538462,616.066667
2,Aquaphobia,385.000000,496.652174,618.421053
3,Avalanche,368.400000,491.000000,615.181818
4,Beach View,372.200000,499.384615,617.411765
5,Beau Recall,373.000000,494.588235,619.920000
6,Beauty Divine,389.333333,515.333333,624.000000
7,Blame the Rider,366.500000,490.000000,614.909091
8,Blossom Trail Miss,378.666667,500.923077,630.500000
9,Cafe Flavor,374.000000,500.307692,616.428571


In [15]:
allworkouts_clean4.to_csv('workout_means')

# Combining final DF

In [16]:
#Merging all the workout dataframes
apr14_final = pd.read_csv('final_merged')
apr7_final = pd.read_csv('final_merged2')
apr6_final = pd.read_csv('final_merged3')

In [17]:
finalmerged_uncleaned = pd.concat([apr14_final, apr7_final, apr6_final], axis=0)

In [18]:
finalmerged_uncleaned.to_csv('final_Times.csv')

In [19]:
finalmerged_uncleaned = pd.read_csv('final_Times.csv').drop('Unnamed: 0', axis=1)

In [20]:
#Adding the percentage of the time the horse places in a race. 
finalmerged_uncleaned["PercentWin"] = (finalmerged_uncleaned["Firsts"] + finalmerged_uncleaned["Seconds"]+ finalmerged_uncleaned["Thirds"])/ (finalmerged_uncleaned["Starts"])
finalmerged_uncleaned.head(2)

,Horse Name,Final_Time,Final_Time_Hund,Gender,Firsts,Seconds,Starts,Thirds,PercentWin
0,Blame the Rider,1:35.11,9511,0,0,0,5,1,0.2
1,Majestic Eagle,1:35.25,9525,0,0,2,4,0,0.5


In [21]:
#Merging the final times df with the #of practice days df
finalmerged_clean = pd.merge(finalmerged_uncleaned,Practice_days,on='Horse Name', how = 'left')
finalmerged_clean.head(2)

,Horse Name,Final_Time,Final_Time_Hund,Gender,Firsts,Seconds,Starts,Thirds,PercentWin,Date
0,Blame the Rider,1:35.11,9511,0,0,0,5,1,0.2,28
1,Majestic Eagle,1:35.25,9525,0,0,2,4,0,0.5,35


In [22]:
#Merging the merged data frame, with the workouts df
finalmerged_clean = pd.merge(finalmerged_clean,allworkouts_clean4,on='Horse Name', how = 'left')
finalmerged_clean.head(2)

,Horse Name,Final_Time,Final_Time_Hund,Gender,Firsts,Seconds,Starts,Thirds,PercentWin,Date,3F,4F,5F
0,Blame the Rider,1:35.11,9511,0,0,0,5,1,0.2,28,366.500000,490.000000,614.909091
1,Majestic Eagle,1:35.25,9525,0,0,2,4,0,0.5,35,386.666667,506.714286,622.666667


In [23]:
#Renaming the columns so they don't confuse the model later.
finalmerged_clean = finalmerged_clean.rename(columns={'3F':'ThreeF'}).rename(columns={'4F':'FourF'}).rename(columns={'5F':'FiveF'}).fillna(0)

In [24]:
finalmerged_clean

,Horse Name,Final_Time,Final_Time_Hund,Gender,Firsts,Seconds,Starts,Thirds,PercentWin,Date,ThreeF,FourF,FiveF
0,Blame the Rider,1:35.11,9511,0,0,0,5,1,0.200000,28,366.500000,490.000000,614.909091
1,Majestic Eagle,1:35.25,9525,0,0,2,4,0,0.500000,35,386.666667,506.714286,622.666667
2,Sellwood,1:35.39,9539,0,0,1,2,1,1.000000,28,365.333333,488.666667,619.333333
3,Cajun Treasure,1:35.39,9539,0,0,0,0,0,0.000000,18,366.500000,496.571429,618.000000
4,Friendly Outthedor,1:35.69,9569,0,0,1,4,0,0.250000,46,373.142857,493.666667,611.500000
5,Pointed,1:35.79,9579,0,0,0,5,0,0.000000,29,367.500000,490.750000,612.769231
6,Dignitaire,1:36.25,9625,0,0,0,3,1,0.333333,32,376.400000,498.571429,623.333333
7,Papa Joe,1:36.71,9671,0,0,0,1,0,0.000000,15,382.666667,503.111111,634.000000
8,Tule Fog,1:34.19,9419,0,1,1,6,1,0.500000,34,381.750000,492.333333,614.727273
9,Lymebyrd,1:34.83,9483,0,2,3,15,1,0.400000,44,369.000000,491.176471,608.421053


In [25]:
finalmerged_clean.shape

(58, 13)

In [26]:
finalmerged_clean.to_csv('finalmerged_clean')